# Sensitivity analysis
For research gaps found, we simulate what would have been the mapping of RCTs within regions across diseases if the misclassification of RCTs towards groups of diseases was corrected, given the sensitivities and specificities of the classifier to identify each group of disease.

To estimate the performances of the classifier for each group of diseases, we dispose a test set with 2,763 trials manually classified towards the 27-class grouping of diseases used in this work. The test set is described at Atal et al. BMC Bioinformatics 2016.

The method used is based on the method presented at Fox et al. Int J Epidemiol 2005.

To do so, for each disease for which we found a local research gap we will:

1. Calculating sensitivity and specificity of the classifier to identify the disease and other studies relevant to the burden of diseases, and the number of success and number of trials to derive beta distributions
2. Doing N=60k times the following simulation
    * Randomly choose a sens and spec based on beta distribution for identifying the disease and identifying another disease (no correlation between sens and spec, neither between disease and another disease both)
    * Derive Positive and Negative Predictive Values (PPV and NPV) for each.
    * Simulate the correction of the classification based on PPVs and NPVs
    * Derive the proportion of RCTs concerning the disease among all RCTs concerning the burden of disease in the region
3. Derive 95% upper-bond simulation interval of the proportion of RCTs concerning the disease among all RCTs concerning the burden of diseases

## 1. Sensitivities and specificities based on test set

In [ ]:
test_set <- read.table("/media/igna/Elements/HotelDieu/Cochrane/MetaMapBurden/Paper_classifier/NCT_data_classified_to28cats.txt")
dim(test_set)

In [ ]:
#We supress injuries from trials concerning the burden of diseases
test_set$GBDnp <- sapply(strsplit(as.character(test_set$GBDnp),"&&"),function(x){paste(x[x!="28"],collapse="&")})
test_set$GBD28 <- sapply(strsplit(as.character(test_set$GBD28),"&"),function(x){paste(x[x!="28"],collapse="&")})

In [ ]:
tst <- strsplit(test_set$GBDnp,"&")
alg <- strsplit(test_set$GBD28,"&")
tst <- lapply(tst,as.numeric)
alg <- lapply(alg,as.numeric)

In [ ]:
source('Evaluation_metrics.R')

In [ ]:
dis <- 1:27
Mgbd <- read.table("/home/igna/Desktop/Programs GBD/Classifier_Trial_GBD/Databases/Taxonomy_DL/GBD_data/GBD_ICD.txt")

In [ ]:
#For each category in 1:27, TP, TN, FP and FN of finding the disease and of finding another disease
set.seed(7212)

dis <- as.character(1:27)

PERF_F  <- data.frame()
for(i in dis){
    ALG <- lapply(alg,function(x){rs <- c()
                                  if(i%in%x) rs <- c(1)
                                  if(sum(setdiff(dis,i)%in%x)!=0) rs <- c(rs,2)
                                  return(rs)
                                      })

    DT <- lapply(tst,function(x){rs <- c()
                                if(i%in%x) rs <- c(1)
                                if(sum(setdiff(dis,i)%in%x)!=0) rs <- c(rs,2)
                                return(rs)
                                    })

    CM <- conf_matrix(ALG,DT,c(1,2))

    PERF <- c(CM[1,],CM[2,])
    PERF_F <- rbind(PERF_F,PERF)
}


In [ ]:
PERF_F <- data.frame(PERF_F)
names(PERF_F) <- paste(rep(c("TP","FP","TN","FN"),2),rep(c("_Dis","_Oth"),each=4),sep="")

In [ ]:
PERF_F$dis <- dis
PERF_F$GBD <- Mgbd$cause_name[-28]

In [ ]:
PERF_F <- PERF_F[,c(9,10,1:8)]

In [ ]:
write.table(PERF_F,'/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Incertitude_mapping/Sp_Sens_evaluation/Sens_and_Spec_per_27disease_data.txt')

## 2. Simulating correction of misclassification

In [1]:
data <- read.table('/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Flowchart/database_all_diseases_final_ok.txt')
N <- nrow(data)

In [2]:
regs <- sort(unique(unlist(strsplit(as.character(data$Regions),"&"))))
LR <- lapply(regs,function(x){1:nrow(data)%in%grep(x,data$Regions)})
LR <- do.call('cbind',LR)

In [3]:
Lgbd <- lapply(as.character(data$GBD28),function(x){as.numeric(unlist(strsplit(x,"&")))})
Lgbd <- lapply(Lgbd,function(x){x[x!=28]})

In [83]:
PERF <- read.table('/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Incertitude_mapping/Sp_Sens_evaluation/Sens_and_Spec_per_27disease_data.txt')

In [5]:
GAPS <- read.table('/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Tables/Local_research_gaps.txt')

In [6]:
#For each disease for which we found a gap, we will simulate the mapping across regions of trials concerning
#the disease or concerning other diseases
dis <- levels(GAPS$Disease)

In [7]:
#NK <- 60000
NK <- 5000
set.seed(7212)

In [8]:
#For each disease
t0 <- proc.time()

for(g in dis){

    PERF_g <- PERF[PERF$GBD==g,]
    
    #which trials concern the disease
    is_dis <- sapply(Lgbd,function(x){PERF_g$dis%in%x})
    #which trials concern another disease
    is_oth <- sapply(Lgbd,function(x){sum(setdiff(1:27,PERF_g$dis)%in%x)>0})

    #PPV et NPVs for finding the disease
    sens_r <- PERF_g$TP_Dis
    sens_n <- PERF_g$TP_Dis + PERF_g$FN_Dis
    spec_r <- PERF_g$TN_Dis
    spec_n <- PERF_g$TN_Dis + PERF_g$FP_Dis
    sens <- rbeta(NK,sens_r+1,sens_n-sens_r+1)
    spec <- rbeta(NK,spec_r+1,spec_n-spec_r+1)

    a_dis <- sum(is_dis)
    b_dis <- N-a_dis
    As <- (a_dis-(1-spec)*N)/(sens - (1-spec))
    Bs <- N-As
    T1 <- sens*As
    T0 <- spec*Bs
    F1 <- (1-spec)*Bs
    F0 <- (1-sens)*As
    PPV_dis <- T1/(T1+F1)
    NPV_dis <- T0/(T0+F0)

    #PPV and NPVs for finding another disease
    sens_r <- PERF_g$TP_Oth
    sens_n <- PERF_g$TP_Oth + PERF_g$FN_Oth
    spec_r <- PERF_g$TN_Oth
    spec_n <- PERF_g$TN_Oth + PERF_g$FP_Oth
    sens <- rbeta(NK,sens_r+1,sens_n-sens_r+1)
    spec <- rbeta(NK,spec_r+1,spec_n-spec_r+1)

    a_oth <- sum(is_oth)
    b_oth <- N-a_oth
    As <- (a_oth-(1-spec)*N)/(sens - (1-spec))
    Bs <- N-As
    T1 <- sens*As
    T0 <- spec*Bs
    F1 <- (1-spec)*Bs
    F0 <- (1-sens)*As
    PPV_oth <- T1/(T1+F1)
    NPV_oth <- T0/(T0+F0)

    #Number of false iterations
    fx_it <- 
          PPV_dis>1 | PPV_dis<0 |
          NPV_dis>1 | NPV_dis<0 |
          PPV_oth>1 | PPV_oth<0 |
          NPV_oth>1 | NPV_oth<0

    #If we have more than 10% of false iterations, we consider that we can not conduct simulations
    if(sum(fx_it)>0.1*NK){ print(paste(c(g,"has", sum(fx_it), "(too many) false iterations"),collapse=" "))
                          next
                          }
    
    print(paste(c(g,"has", sum(fx_it), "false iterations"),collapse=" "))

    PPV_dis <- PPV_dis[!fx_it]
    NPV_dis <- NPV_dis[!fx_it]
    PPV_oth <- PPV_oth[!fx_it]
    NPV_oth <- NPV_oth[!fx_it]


    L <- list()
    #Simulation: reclassifying each trial
        for(k in 1:sum(!fx_it)){

            AR <- matrix(0, nrow=length(regs), ncol=2)
            tp_dis <- runif(a_dis)
            tn_dis <- runif(b_dis)
            recl_dis <- is_dis
            recl_dis[recl_dis==TRUE][tp_dis>PPV_dis[k]] <- FALSE
            recl_dis[recl_dis==FALSE][tn_dis>NPV_dis[k]] <- TRUE
            #Rq: we count all trials (even those with more than 3 diseases)
            #it is a conservative choice
            rt <- as.numeric(recl_dis)

            AR[,1] <- apply(LR[recl_dis,],2,sum)

            #Oth_dis
            tp_oth <- runif(a_oth)
            tn_oth <- runif(b_oth)
            recl_oth <- is_oth
            recl_oth[recl_oth==TRUE][tp_oth>PPV_oth[k]] <- FALSE
            recl_oth[recl_oth==FALSE][tn_oth>NPV_oth[k]] <- TRUE
            rt <- rt + as.numeric(recl_oth)

            AR[,2] <- apply(LR[rt!=0,],2,sum)

            L[[k]] <- AR

        }
   
    T <- do.call('rbind',L)
    write.table(T,paste(c("/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Incertitude_mapping/Simulations/Res_simul27_",as.character(PERF_g$dis),".txt"),collapse=""))

}

t1 <- proc.time()
    
print(t1-t0)/60

[1] "Cardiovascular and circulatory diseases has 0 false iterations"
[1] "Congenital anomalies has 693 (too many) false iterations"
[1] "Diarrhea, lower respiratory infections, meningitis, and other common infectious diseases has 0 false iterations"
[1] "Neglected tropical diseases excluding malaria has 0 false iterations"
[1] "Neonatal disorders has 0 false iterations"
[1] "Nutritional deficiencies has 1 false iterations"
[1] "Sexually transmitted diseases excluding HIV has 504 (too many) false iterations"
[1] "Tuberculosis has 33 false iterations"
    user   system  elapsed 
2274.598    3.061 2280.720 


       user      system     elapsed 
37.90996667  0.05101667 38.01200000 

For 60,000 simulations it will take 20h approx

## 3. Deriving 95% upper bound simulation intervals

In [69]:
GAPS <- read.table('/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Tables/Local_research_gaps.txt')

In [70]:
regs <- levels(GAPS$Region)

In [71]:
Mgbd <- read.table("/home/igna/Desktop/Programs GBD/Classifier_Trial_GBD/Databases/Taxonomy_DL/GBD_data/GBD_ICD.txt")

In [72]:
sims <- list.files("/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Incertitude_mapping/Simulations")

In [73]:
#diseases for which we have a simulation
dis <- as.character(Mgbd$cause_name[as.numeric(substr(sims,13,nchar(sims)-4))])

In [74]:
GAPS$Sim_RCTs <- NA

In [75]:
for(g in dis){

    T <- tryCatch(read.table(paste(c("/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Incertitude_mapping/Simulations/Res_simul27_",
                            as.character(which(Mgbd$cause_name==g)),".txt"),collapse="")),error=NULL)

    if(length(T)!=0){
        M <- matrix(T[,1]/T[,2],ncol=7,byrow=TRUE)
        SIM <- apply(M,2,function(x){quantile(x,0.95)})
        SIM <- data.frame(SIM)
        GAPS$Sim_RCTs[GAPS$Disease==g] <- SIM$SIM[regs%in%GAPS$Region[GAPS$Disease==g]]
    }
    
}


In [76]:
GAPS$Robust <- FALSE

In [77]:
GAPS$Robust[2*GAPS$Sim_RCTs<GAPS$Pr_GBD] <- TRUE

In [78]:
GAPS[,c(7,9,10)] <- round(100*GAPS[,c(7,9,10)],digits=2)

In [96]:
GAPS[GAPS$Robust,c(4,3,7,10,9)]

,Region,Disease,Pr_RCT,Sim_RCTs,Pr_GBD
1,"Central Europe, Eastern Europe, and Central Asia",Cardiovascular and circulatory diseases,11.27,13.77,35.09
3,"Central Europe, Eastern Europe, and Central Asia",Tuberculosis,0.3,0.44,1.83
6,Latin America and Caribbean,Neonatal disorders,0.86,3.42,8.72
9,North Africa and Middle East,"Diarrhea, lower respiratory infections, meningitis, and other common infectious diseases",2.91,4.04,10.55
10,North Africa and Middle East,Neonatal disorders,2.08,4.49,10.06
12,South Asia,"Diarrhea, lower respiratory infections, meningitis, and other common infectious diseases",7.36,8.05,21.43
13,South Asia,Neonatal disorders,1.77,4.33,16.52
14,South Asia,Tuberculosis,1.11,1.21,3.87
17,"Southeast Asia, East Asia and Oceania",Tuberculosis,0.73,0.84,2.84
19,Sub-Saharian Africa,"Diarrhea, lower respiratory infections, meningitis, and other common infectious diseases",5.8,6.74,22.93


In [97]:
GAPS[!GAPS$Robust,c(4,3,7,10,9)]

,Region,Disease,Pr_RCT,Sim_RCTs,Pr_GBD
2,"Central Europe, Eastern Europe, and Central Asia",Neonatal disorders,0.54,3.03,2.56
4,High-income,Neonatal disorders,0.82,3.48,1.65
5,Latin America and Caribbean,Congenital anomalies,1.15,NA,3.13
7,Latin America and Caribbean,Nutritional deficiencies,1.64,2.34,3.76
8,North Africa and Middle East,Congenital anomalies,1.69,NA,4.98
11,North Africa and Middle East,Nutritional deficiencies,1.54,2.29,4.12
15,"Southeast Asia, East Asia and Oceania",Neglected tropical diseases excluding malaria,0.67,1.11,1.55
16,"Southeast Asia, East Asia and Oceania",Neonatal disorders,0.49,3.14,6.06
18,Sub-Saharian Africa,Congenital anomalies,0.62,NA,1.33
21,Sub-Saharian Africa,Sexually transmitted diseases excluding HIV,0.35,NA,1.27


In [85]:
PERF[PERF$GBD=="Congenital anomalies",]

,dis,GBD,TP_Dis,FP_Dis,TN_Dis,FN_Dis,TP_Oth,FP_Oth,TN_Oth,FN_Oth
23,23,Congenital anomalies,22,34,2706,1,2121,205,275,162


In [86]:
g <- "Congenital anomalies"

In [87]:
PERF_g <- PERF[PERF$GBD==g,]
    
    #which trials concern the disease
    is_dis <- sapply(Lgbd,function(x){PERF_g$dis%in%x})
    #which trials concern another disease
    is_oth <- sapply(Lgbd,function(x){sum(setdiff(1:27,PERF_g$dis)%in%x)>0})

    #PPV et NPVs for finding the disease
    sens_r <- PERF_g$TP_Dis
    sens_n <- PERF_g$TP_Dis + PERF_g$FN_Dis
    spec_r <- PERF_g$TN_Dis
    spec_n <- PERF_g$TN_Dis + PERF_g$FP_Dis
    sens <- rbeta(NK,sens_r+1,sens_n-sens_r+1)
    spec <- rbeta(NK,spec_r+1,spec_n-spec_r+1)



In [88]:
sens_r

[1] 22

In [89]:
sens

[1] 0.959 0.952 0.944 0.895 0.962 0.891 0.895 0.943 0.968 0.950 0.936 0.827
  [13] 0.834 0.963 0.956 0.852 0.993 0.894 0.904 0.978 0.912 0.916 0.982 0.795
  [25] 0.947 0.845 0.977 0.823 0.897 0.922 0.846 0.972 0.971 0.963 0.865 0.943
  [37] 0.877 0.794 0.887 0.892 0.985 0.984 0.905 0.897 0.964 0.904 0.978 0.954
  [49] 0.946 0.958 0.922 0.982 0.862 0.923 0.948 0.958 0.927 0.904 0.912 0.940
  [61] 0.893 0.957 0.983 0.972 0.904 0.906 0.991 0.814 0.931 0.928 0.745 0.989
  [73] 0.968 0.947 0.952 0.847 0.971 0.940 0.835 0.897 0.896 0.931 0.862 0.813
  [85] 0.910 0.921 0.976 0.985 0.911 0.976 0.928 0.920 0.935 0.926 0.838 0.979
  [97] 0.874 0.832 0.960 0.972 0.990 0.857 0.953 0.940 0.877 0.993 0.917 0.872
 [109] 0.877 0.918 0.858 0.882 0.939 0.981 0.982 0.833 0.881 0.936 0.919 0.948
 [121] 0.861 0.919 0.950 0.928 0.945 0.821 0.953 0.907 0.920 0.960 0.934 0.974
 [133] 0.912 0.947 0.890 0.950 0.888 0.934 0.965 0.984 0.969 0.982 0.910 0.813
 [145] 0.841 0.991 0.940 0.936 0.975 0.974 0.950 0.950 0.960 0.948 0.922 0.856
 [157] 0.940 0.943 0.967 0.929 0.965 0.992 0.843 0.907 0.953 0.994 0.974 0.933
 [169] 0.952 0.945 0.867 0.984 0.919 0.951 0.967 0.846 0.973 0.983 0.932 0.973
 [181] 0.915 0.989 0.972 0.933 0.832 0.931 0.950 0.878 0.906 0.908 0.933 0.834
 [193] 0.982 0.851 0.978 0.963 0.906 0.899 0.861 0.829 0.977 0.977 0.923 0.976
 [205] 0.961 0.914 0.880 0.994 0.875 0.915 0.841 0.908 0.849 0.889 0.924 0.964
 [217] 0.922 0.946 0.969 0.948 0.894 0.919 0.939 0.921 0.956 0.962 0.959 0.897
 [229] 0.911 0.967 0.995 0.897 0.867 0.980 0.909 0.951 0.937 0.962 0.965 0.884
 [241] 0.882 0.926 0.970 0.963 0.865 0.893 0.991 0.957 0.944 0.802 0.961 0.904
 [253] 0.902 0.945 0.971 0.922 0.935 0.974 0.976 0.825 0.814 0.780 0.875 0.882
 [265] 0.962 0.943 0.952 0.916 0.939 0.897 0.950 0.934 0.958 0.986 0.777 0.987
 [277] 0.781 0.945 0.960 0.888 0.933 0.955 0.932 0.853 0.979 0.900 0.928 0.919
 [289] 0.748 0.886 0.778 0.944 0.874 0.922 0.902 0.851 0.896 0.839 0.981 0.955
 [301] 0.967 0.948 0.955 0.939 0.914 0.961 0.902 0.979 0.858 0.878 0.840 0.957
 [313] 0.986 0.949 0.902 0.985 0.949 0.934 0.947 0.918 0.909 0.947 0.984 0.860
 [325] 0.851 0.909 0.983 0.988 0.890 0.993 0.634 0.954 0.956 0.970 0.904 0.770
 [337] 0.851 0.864 0.926 0.945 0.942 0.898 0.924 0.888 0.942 0.886 0.956 0.968
 [349] 0.910 0.918 0.906 0.885 0.954 0.980 0.910 0.951 0.941 0.926 0.922 0.964
 [361] 0.986 0.884 0.750 0.865 0.920 0.967 0.854 0.961 0.930 0.880 0.865 0.964
 [373] 0.917 0.967 0.896 0.865 0.861 0.964 0.892 0.940 0.981 0.976 0.952 0.900
 [385] 0.783 0.895 0.973 0.878 0.912 0.919 0.941 0.957 0.884 0.916 0.931 0.889
 [397] 0.808 0.753 0.951 0.931 0.955 0.954 0.965 0.902 0.881 0.752 0.855 0.936
 [409] 0.865 0.908 0.811 0.970 0.934 0.820 0.981 0.965 0.940 0.932 0.870 0.978
 [421] 0.911 0.914 0.893 0.888 0.904 0.912 0.916 0.932 0.964 0.983 0.980 0.941
 [433] 0.969 0.824 0.971 0.758 0.928 0.939 0.880 0.931 0.962 0.904 0.909 0.855
 [445] 0.947 0.981 0.924 0.820 0.980 0.978 0.927 0.844 0.941 0.972 0.960 0.942
 [457] 0.907 0.975 0.953 0.930 0.973 0.959 0.967 0.944 0.950 0.874 0.913 0.974
 [469] 0.886 0.918 0.913 0.903 0.953 0.848 0.937 0.882 0.933 0.955 0.917 0.950
 [481] 0.699 0.908 0.928 0.914 0.887 0.933 0.871 0.813 0.771 0.887 0.797 0.929
 [493] 0.970 0.973 0.924 0.947 0.982 0.841 0.965 0.937 0.965 0.967 0.933 0.930
 [505] 0.943 0.848 0.912 0.932 0.945 0.950 0.906 0.974 0.838 0.897 0.945 0.919
 [517] 0.861 0.948 0.942 0.937 0.936 0.887 0.906 0.897 0.957 0.903 0.948 0.869
 [529] 0.948 0.945 0.981 0.984 0.919 0.937 0.944 0.938 0.931 0.924 0.972 0.977
 [541] 0.934 0.958 0.892 0.857 0.886 0.982 0.780 0.873 0.961 0.946 0.960 0.833
 [553] 0.830 0.965 0.818 0.913 0.929 0.851 0.850 0.980 0.918 0.947 0.934 0.919
 [565] 0.856 0.995 0.704 0.898 0.948 0.927 0.749 0.947 0.982 0.981 0.901 0.909
 [577] 0.950 0.785 0.961 0.968 0.989 0.934 0.992 0.869 0.950 0.895 0.855 0.940
 [589] 0.912 0.889 0.878 0.803 0.946 0.971 0.963 0.958 0.920 0.915 0.981 0.903
 [601] 0.921 0.849 0.965 0.986 0.903 0.866 0.942 0.93

ERROR: Error in grDevices::replayPlot(obj): X11 font -adobe-helvetica-%s-%s-*-*-%d-*-*-*-*-*-*-*, face 2 at size 14 could not be loaded


Plot with title “Histogram of sens”

In [91]:
x11()

In [93]:
hist(sens)

ERROR: Error in grDevices::replayPlot(obj): X11 font -adobe-helvetica-%s-%s-*-*-%d-*-*-*-*-*-*-*, face 2 at size 14 could not be loaded


Plot with title “Histogram of sens”